# Predicting Home Loans' Repayment and Analysis of Unbanked Population

The unbanked population, consumers without adequate credit history, will go to an untrustworthy lender whom could take unfair advantage of their situation because reputable banks will not provide them with loans. 
However, there could be other ways to determine the credit worthiness of applicants. The vision of Home Credit Group is to provide fair and equal services to a non-traditional home loan applicant. 


1) Determine if the applicant will likely repay their loan by using previous credit information via the Home Credit Group's databases and the Credit Bureau

2) Find non-traditional methods to determine the credit worthiness of an applicant (unbanked

In [1]:
import pandas as pd
import numpy as np
import pickle

#Allows to see up to 500 columns within notebook
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

# Load datasets into dataframes

In [2]:
df_application_train = pd.read_csv('application_train.csv',header=0,error_bad_lines=False,nrows=10000)
df_previous_application = pd.read_csv('previous_application.csv',header=0,error_bad_lines=False,nrows=50000)
df_installment_payments = pd.read_csv('installments_payments.csv',header=0,error_bad_lines=False,nrows=50000)
df_bureau = pd.read_csv('bureau.csv',header=0,error_bad_lines=False,nrows=50000)

#I do not use the following datasets within this notebook. However, it could be useful at a later time.
#df_bureau_bal = pd.read_csv('bureau_balance.csv',header=0,error_bad_lines=False,nrows=5000)
#df_pos_cash_bal = pd.read_csv('POS_CASH_balance.csv',header=0,error_bad_lines=False,nrows=5000)
#df_credit_card_bal = pd.read_csv('credit_card_balance.csv',header=0,error_bad_lines=False,nrows=5000)

#dflist = [df_application_train,df_previous_application,df_installment_payments,df_bureau,df_bureau_bal,df_pos_cash_bal,df_credit_card_bal]


# Previous Application Dataset

Started with Previous Application Dataset because it had historical information about the application, so I believe it is extremely important. 

In [3]:
df_previous_application.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


#Cleaning Dataset

1. FLAG_LAST_APPL_PER_CONTRACT: If this column had the value of 'N', then it was more than likely a mistake or duplicate of another application. My thought is that this maybe unnecessary data to the purpose of this project.

2. AMT_APPLICATION: This column states how much the applicant has asked for from the bank. 

3. AMT_CREDIT: This column states how much the bank as credited the applicant.

My thought is if (2) and (3) are 0, then the application is incomplete and it would not provide me with additional information about the application. Thus I should remove these rows from my dataset.
        

In [4]:
#df_previous_application = df_previous_application.drop(df_previous_application[df_previous_application['FLAG_LAST_APPL_PER_CONTRACT']=='N'].index)
#df_previous_application = df_previous_application.drop(df_previous_application[(df_previous_application['AMT_APPLICATION']==0.0) & (df_previous_application['AMT_CREDIT']==0.0)].index)



In the next cell, the 'RATE_DOWN_PAYMENT','RATE_INTEREST_PRIMARY', and 'RATE_INTEREST_PRIVILEGED' columns has 'NaN' values. Because I will be aggregating the columns to merge them into the main dataset (application_train), I have replaced the NaN values with 0. 

In [5]:
# Function to calculate missing values by column# Funct 
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [6]:
missing_values = missing_values_table(df_previous_application)
missing_values

Your selected dataframe has 37 columns.
There are 15 columns that have missing values.


,Missing Values,% of Total Values
RATE_INTEREST_PRIMARY,49835,99.7
RATE_INTEREST_PRIVILEGED,49835,99.7
AMT_DOWN_PAYMENT,25198,50.4
RATE_DOWN_PAYMENT,25198,50.4
NAME_TYPE_SUITE,24243,48.5
DAYS_FIRST_DRAWING,19161,38.3
DAYS_FIRST_DUE,19161,38.3
DAYS_LAST_DUE_1ST_VERSION,19161,38.3
DAYS_LAST_DUE,19161,38.3
DAYS_TERMINATION,19161,38.3


The interest rates previously provided to applicants had many incomplete values. I filled it with 0, which states that they did not provide an interest rate. 

Previous application dataset has all of the previous application available. Currently, it has a 1:many relationship to the application dataset, so I will need to aggregate the columns before merging. Below is where I aggreagated the previous_application dataset by using index 'SK_ID_CURR'. The purpose of this is so I can provide a 1:1 relationship to the train/test dataset, while limit the columns created.

In [7]:
df_prevapp_pivot = pd.pivot_table(df_previous_application,values=df_previous_application.columns,index='SK_ID_CURR',
                                  aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())), 'NAME_CONTRACT_TYPE': (lambda x: len(x.unique())), 
                                           'AMT_ANNUITY':'sum', 'AMT_APPLICATION': 'sum','AMT_CREDIT':'sum', 'AMT_DOWN_PAYMENT':'sum', 
                                           'AMT_GOODS_PRICE':'sum','WEEKDAY_APPR_PROCESS_START':(lambda x: len(x.unique())), 
                                           'HOUR_APPR_PROCESS_START':np.mean,'FLAG_LAST_APPL_PER_CONTRACT':(lambda x: len(x.unique())), 
                                           'NFLAG_LAST_APPL_IN_DAY':(lambda x: len(x.unique())),'RATE_DOWN_PAYMENT':np.mean, 
                                           'RATE_INTEREST_PRIMARY':np.mean,'RATE_INTEREST_PRIVILEGED':np.mean, 
                                           'NAME_CASH_LOAN_PURPOSE':(lambda x: len(x.unique())),
                                           'NAME_CONTRACT_STATUS':(lambda x:x.map({'Approved':1,'Canceled':0,'Refused':0,'Unused offer':1}).mean()), 
                                           'DAYS_DECISION':np.mean, 'NAME_PAYMENT_TYPE':(lambda x: len(x.unique())),
                                           'CODE_REJECT_REASON':(lambda x: len(x.unique())),'NAME_TYPE_SUITE':(lambda x: len(x.unique())), 
                                           'NAME_CLIENT_TYPE':(lambda x: len(x.unique())),'NAME_GOODS_CATEGORY':(lambda x: len(x.unique())), 
                                           'NAME_PORTFOLIO':(lambda x: len(x.unique())), 'NAME_PRODUCT_TYPE':(lambda x: len(x.unique())),
                                           'CHANNEL_TYPE':(lambda x: len(x.unique())), 'SELLERPLACE_AREA':(lambda x: len(x.unique())), 
                                           'NAME_SELLER_INDUSTRY':(lambda x: len(x.unique())),'CNT_PAYMENT':'sum', 
                                           'NAME_YIELD_GROUP':(lambda x: len(x.unique())), 'PRODUCT_COMBINATION':(lambda x: len(x.unique())),
                                           'DAYS_FIRST_DRAWING':'sum', 'DAYS_FIRST_DUE':'sum', 'DAYS_LAST_DUE_1ST_VERSION':'sum',
                                           'DAYS_LAST_DUE':'sum', 'DAYS_TERMINATION':'sum', 'NFLAG_INSURED_ON_APPROVAL':np.mean})

# Merging previous_application dataset into application_train dataset 

In [8]:
df = df_application_train.merge(df_prevapp_pivot,on =['SK_ID_CURR'],suffixes=('_app_train','_prev_app'),how='left')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 158 entries, SK_ID_CURR to WEEKDAY_APPR_PROCESS_START_prev_app
dtypes: float64(101), int64(41), object(16)
memory usage: 12.1+ MB


# Aggreagated Bureau & Installment datasets and then merged into dataset

In [10]:
df_bureau_pivot = pd.pivot_table(df_bureau,values=df_bureau.columns,index='SK_ID_CURR',
                                 aggfunc={'SK_ID_BUREAU':'count','CREDIT_ACTIVE':'count',
                                          'CREDIT_CURRENCY':(lambda x: len(x.unique())),'DAYS_CREDIT':np.min,
                                          'CREDIT_DAY_OVERDUE':'sum','DAYS_CREDIT_ENDDATE':'sum', 
                                          'DAYS_ENDDATE_FACT':'sum', 'AMT_CREDIT_MAX_OVERDUE':'sum', 
                                          'CNT_CREDIT_PROLONG':'sum', 'AMT_CREDIT_SUM':'sum','AMT_CREDIT_SUM_DEBT':'sum', 
                                          'AMT_CREDIT_SUM_LIMIT':'sum', 'AMT_CREDIT_SUM_OVERDUE':'sum',
                                          'CREDIT_TYPE':(lambda x: len(x.unique())), 'DAYS_CREDIT_UPDATE':np.min, 
                                          'AMT_ANNUITY':'sum'})

In [11]:
df = df.merge(df_bureau_pivot,on =['SK_ID_CURR'],suffixes=('_curr','_bur'),how='left')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 174 entries, SK_ID_CURR to SK_ID_BUREAU
dtypes: float64(117), int64(41), object(16)
memory usage: 13.4+ MB


# Merging Installment Payment Dataset


AMT_INSTAL_PAY_DIFF: I believe that knowing how much the applicant has paid back in the past would be a good indicator for the future

NUM_INSTAL_VERSION_NUM_DIFF: I believe knowing how fast or slow the previous load was paid would also be a good indicator.

DAY_INSTAL_ENTRY_DIFF: Similar to NUM_INSTAL_VERSION_NUM_DIFF

In [13]:
df_installment_payments_pivot = pd.pivot_table(df_installment_payments,values=df_installment_payments.columns,index='SK_ID_CURR',
                                               aggfunc={'SK_ID_PREV':(lambda x: len(x.unique())),'NUM_INSTALMENT_VERSION':'sum', 
                                                        'NUM_INSTALMENT_NUMBER':'sum','DAYS_INSTALMENT':'sum', 'DAYS_ENTRY_PAYMENT':'sum', 
                                                        'AMT_INSTALMENT':'sum','AMT_PAYMENT':'sum'})                                                                                                                      

In [14]:
#created difference columns to provide some potential useful columns to the model
df_installment_payments_pivot['AMT_INSTAL_PAY_DIFF'] = df_installment_payments_pivot['AMT_INSTALMENT']-df_installment_payments_pivot['AMT_PAYMENT']
df_installment_payments_pivot['NUM_INSTAL_VERSION_NUM_DIFF'] = df_installment_payments_pivot['NUM_INSTALMENT_VERSION']-df_installment_payments_pivot['NUM_INSTALMENT_NUMBER']
df_installment_payments_pivot['DAY_INSTAL_ENTRY_DIFF'] = df_installment_payments_pivot['DAYS_INSTALMENT']-df_installment_payments_pivot['DAYS_ENTRY_PAYMENT']


In [15]:
df = df.merge(df_installment_payments_pivot,on =['SK_ID_CURR'],suffixes=('_curr2','_instpay'),how='left')

In [16]:
#Many current applicants do not have historical information. 
#Should test the differences between the 2 groups.

df['BANKED']= df.SK_ID_CURR.isin(df_bureau.SK_ID_CURR)
df['BANKED']=df.BANKED.map({True:1,False:0})

In [17]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 185 entries, SK_ID_CURR to BANKED
dtypes: float64(127), int64(42), object(16)
memory usage: 14.2+ MB
None


# Feature Elimination via SelectFromModel & LassoCV

I currently have 185 columns with 16 Categorical columns. I will need to cut the size of dataframe to prevent overfitting the models. I have used the SelectFromModel as the Meta-Transformer and LassoCV as the classifier. As a result, I have decreased my overall dataframe size by ~40%.  


In [18]:
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.linear_model import LassoCV


df_dumb = pd.get_dummies(df.select_dtypes('object'),dummy_na=True)
df_concat_dumb = pd.concat([df.select_dtypes(exclude=['object']),df_dumb],axis=1)

#Go to null value used is 365423, so I decided to use it across the df
df_concat_dumb.fillna(value=365423,inplace=True)
df_concat_dumb.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Columns: 320 entries, SK_ID_CURR to EMERGENCYSTATE_MODE_nan
dtypes: float64(127), int64(42), uint8(151)
memory usage: 14.4 MB


In [19]:
df_test = df_concat_dumb.loc[:, df_concat_dumb.columns != 'TARGET']

y = df_concat_dumb.TARGET


In [20]:
 from sklearn.svm import SVR
clf = LassoCV()
model = SelectFromModel(clf,threshold=.0000000001)
model.fit(df_test,y)

passed_feature_test = df_test.columns[model.get_support()].tolist()
len(passed_feature_test)

/home/nervster/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/home/nervster/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


87

In [21]:
passed_feature_test.append('TARGET')

df_feateliminated = df_concat_dumb[passed_feature_test]
len(passed_feature_test)

88

In [22]:
df_feateliminated.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 88 columns):
AMT_INCOME_TOTAL                10000 non-null float64
AMT_CREDIT_app_train            10000 non-null float64
AMT_ANNUITY_app_train           10000 non-null float64
AMT_GOODS_PRICE_app_train       10000 non-null float64
DAYS_BIRTH                      10000 non-null int64
DAYS_EMPLOYED                   10000 non-null int64
DAYS_REGISTRATION               10000 non-null float64
DAYS_ID_PUBLISH                 10000 non-null int64
OWN_CAR_AGE                     10000 non-null float64
EXT_SOURCE_1                    10000 non-null float64
EXT_SOURCE_3                    10000 non-null float64
APARTMENTS_AVG                  10000 non-null float64
BASEMENTAREA_AVG                10000 non-null float64
YEARS_BEGINEXPLUATATION_AVG     10000 non-null float64
YEARS_BUILD_AVG                 10000 non-null float64
COMMONAREA_AVG                  10000 non-null float64
ELEVATORS_AVG     

# Convert Dataframe into Pickle File for further Exploration

In [23]:
df_feateliminated.to_pickle('df_pickle.pkl')